In [1]:
import re
import numpy as np
import pandas as pd
import os
import PIL
import random
import shutil
import matplotlib.pyplot as plt
import PIL
import sys
import cv2
from tqdm import tqdm
from PIL import Image, ImageMath

from skimage.transform import rescale, resize, downscale_local_mean
from img_processing_256 import rename, random_flip_img_train
%matplotlib inline

In [2]:
def remove_margins(image_arr, margin=20):
	"""
	function to trim plxels off all sides of an image
	"""
	h, w = image_arr.shape
	new_image = image_arr[margin:h-margin,margin:w-margin]
	return new_image

In [3]:
def mask_img2(mask_path, full_image_arr, slice_size=512):

	mask = PIL.Image.open(mask_path)
	print(mask.size)
#	if half:
#		print("half")
#		h, w = mask.size
#		new_size = ( h // 2, w // 2)
#		mask.thumbnail(new_size, PIL.Image.ANTIALIAS)
#	print(mask.size)
    
	mask_arr = np.array(mask)
	mask_arr = mask_arr[:,:]
    
#	plt.imshow(mask_arr, cmap='gray')
    
#	if np.sum(np.sum(full_image_arr >= 225)) > 20000:
#		#plt.imshow(full_image_arr, cmap='gray')
#		full_image_arr = remove_margins(full_image_arr)
#		mask_arr = remove_margins(mask_arr)
        
		#plt.imshow(full_image_arr, cmap='gray')
        
#		if output:
#			print("Trimming borders", mask_path)
			
	# The maks size must be same as the full image size
#	if mask_arr.shape != full_image_arr.shape:
		# see if the ratios are the same
#		mask_ratio = mask_arr.shape[0] / mask_arr.shape[1]
#		image_ratio = full_image_arr.shape[0] / full_image_arr.shape[1]
		
#		if abs(mask_ratio - image_ratio) <=  1e-03:
#			if output:
#				print("Mishaped mask, resizing mask", mask_path)
			
			# reshape the mask to match the image
			#mask_arr = imresize(mask_arr, full_image_arr.shape)
#			mask_arr = np.array(Image.fromarray(mask_arr).resize(full_image_arr.shape))
#		else:
#			if output:
#				print("Mask shape:", mask_arr.shape)
#				print("Image shape:", full_image_arr.shape)
#			print("Mask shape doesn't match image!", mask_path)
#			return 0, 0, False, full_image_arr, 0
	
	# find the borders
	mask_mask = mask_arr == 255

	# check whether each row or column have a white pixel
	cols = np.sum(mask_mask, axis=0)
	rows = np.sum(mask_mask, axis=1)

	# check corners
	first_col = np.argmax(cols > 0)
	last_col = mask_arr.shape[1] - np.argmax(np.flip(cols, axis=0) > 0)
	center_col = int((first_col + last_col) / 2)

	first_row = np.argmax(rows > 0)
	last_row = mask_arr.shape[0] - np.argmax(np.flip(rows, axis=0) > 0)
	center_row = int((first_row + last_row) / 2)
	
	col_size = last_col - first_col
	row_size = last_row - first_row
	
	mask_size = [row_size, col_size]
#	print(mask_size)
	# When a mask size is bigger than a slice
	too_big = False
	
	if (last_col - first_col > slice_size + 30) or (last_row - first_row > slice_size + 30):
		too_big = True
#	print("center_row:", center_row)
#	print("center_col:", center_col)
	return center_row, center_col, too_big, mask_size

In [4]:
def create_patches(roi_size, mask_dir, img_dir, Lbls, size=256, debug=True):
    patch_list = []
    Lbl_list = []
    FN_list = []
    
    full_size = 512
    masks = os.listdir(mask_dir)
    counter = 0
    if debug is None:
        progress(counter, len(masks), 'WORKING')
    for mask in tqdm(masks):
        counter += 1
        if debug is None:
            progress(counter, len(masks), mask)
        base_img_file = mask[:-6] + ".png"
        full_img = PIL.Image.open(img_dir+"/"+base_img_file)
#        try:
#            full_img = PIL.Image.open(img_dir + "/" + base_img_file + '.png')
#        except:
#            try:
#                full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000000.png')
#            except:
#                try:
#                    full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000001.png')
#                except:
#                    print("Error FileNotFound:", base_img_file)
#                    continue
        compare = base_img_file[14:]
        #print(compare)
        try:
            Lbl = Lbls.loc[compare]['Class']
        except:
            print("Error LabelNotFound", base_img_file)
            continue
            
        full_img_arr = np.array(full_img)[:,:]
        ctr_row, ctr_col, too_big, mask_size = mask_img2(mask_dir + "/" + mask,full_img_arr)
        img_h, img_w = full_img_arr.shape
        try:
            mask_H = mask_size[0]
            mask_W = mask_size[1]
            roi_size = np.max([mask_H, mask_W])
            if debug:
                print("Mask", mask, " Height:", mask_H, "Width:", mask_W)
        except:
            print("Mask Size Error:", mask_size, "for", mask)
        # Record roi size for DDSM image crop
        roi_sizes.append(roi_size)
        if (ctr_row == 0) and (ctr_col == 0):
            print("Error, skipping", mask)
            continue
        """
        Extract the ROI depending on it's size
        If the ROI is smaller than a slice extract it with some padding
        """
        
#        if roi_size < full_size:

        if(mask_H > mask_W):
            mask_W = mask_H
        else:
            mask_H = mask_W
        
        adj_mask_H = mask_H*1.15
        adj_mask_W = mask_W*1.15
        
        
        start_row = np.max([ctr_row - 512, 0])
        if start_row == 0 :
            end_row = 1024
        else :
            end_row = start_row + 1024
            
        if end_row > img_h:
            end_row = img_h
            start_row = img_h - 1024
            
        start_col = np.max([ctr_col - 512, 0])
        if start_col == 0:
            end_col = 1024
        else :
            end_col = start_col + 1024
        if end_col > img_w:
            end_col = img_w
            start_col = img_w - 1024
       
        end_row = int(end_row)
        end_col = int(end_col)
        start_row = int(start_row)
        start_col = int(start_col)
        print("start_row",start_row)
        print("end_row",end_row)
        print("start_col",start_col)
        print("end_col",end_col)
        
        roi_img = random_flip_img_train(full_img_arr[start_row:end_row, start_col:end_col])
        
        print("roi_img.shape",roi_img.shape)    
        
        roi_img = roi_img.astype('float32')
        
        roi_img = cv2.resize(roi_img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        roi_img = roi_img.reshape((256, 256, 1))
        #print(roi_img.shape[0])
        #print(roi_img.shape[1])
        
        
        #print(roi_img)
#        patch_1 = crop_img2(roi_img)
#        patch_2 = crop_img2(roi_img)
#        patch_3 = crop_img2(roi_img)
        
        patch_list.append(roi_img)
        Lbl_list.append(Lbl)
        FN_list.append(base_img_file) 
            
#        if (patch_1.shape[0] == size) and (patch_1.shape[1] == size):
#            patch_list.append(patch_1)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
#        if (patch_2.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_2)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
        
#        if (patch_3.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_3)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
    return np.array(patch_list), np.array(Lbl_list), np.array(FN_list), roi_sizes

def create_patches(roi_size, mask_dir, img_dir, Lbls, size=256, debug=True):
    patch_list = []
    Lbl_list = []
    FN_list = []
    
    full_size = 512
    masks = os.listdir(mask_dir)
    counter = 0
    if debug is None:
        progress(counter, len(masks), 'WORKING')
    for mask in tqdm(masks):
        counter += 1
        if debug is None:
            progress(counter, len(masks), mask)
        base_img_file = mask[:-6] + ".png"
        full_img = PIL.Image.open(img_dir+"/"+base_img_file)
        
        #compare = base_img_file[14:]
        compare = base_img_file[10:]
        try:
            Lbl = Lbls.loc[compare]['Class']
        except:
            print("Error LabelNotFound", base_img_file)
            continue
            
        full_img_arr = np.array(full_img)[:,:]
        ctr_row, ctr_col, too_big, mask_size = mask_img2(mask_dir + "/" + mask,full_img_arr)
        img_h, img_w = full_img_arr.shape
        try:
            mask_H = mask_size[0]
            mask_W = mask_size[1]
            roi_size = np.max([mask_H, mask_W])
            if debug:
                print("Mask", mask, " Height:", mask_H, "Width:", mask_W)
        except:
            print("Mask Size Error:", mask_size, "for", mask)
        roi_sizes.append(roi_size)
        if (ctr_row == 0) and (ctr_col == 0):
            print("Error, skipping", mask)
            continue
        """
        Extract the ROI depending on it's size
        If the ROI is smaller than a slice extract it with some padding
        """
        
        adj_mask_H = mask_H*1.15
        adj_mask_W = mask_W*1.15
        
        start_row = np.max([ctr_row - (adj_mask_H // 2), 0])
        end_row = start_row + adj_mask_H
        if end_row > img_h:
            end_row = img_h
            start_row = img_h - adj_mask_H
            
        start_col = np.max([ctr_col - (adj_mask_W // 2), 0])
        end_col = start_col + adj_mask_W
        if end_col > img_w:
            end_col = img_w
            start_col = img_w - adj_mask_W
        end_row = int(end_row)
        end_col = int(end_col)
        start_row = int(start_row)
        start_col = int(start_col)
        roi_img = random_flip_img_train(full_img_arr[start_row:end_row, start_col:end_col])
        
        patch_list.append(roi_img)
        Lbl_list.append(Lbl)
        FN_list.append(base_img_file)
                
    return np.array(patch_list), np.array(Lbl_list), np.array(FN_list), roi_sizes

In [5]:
#Calc_training
train_labels = pd.read_pickle("label old ver/train_label.pkl")
train_labels['IMAGE_NAME2'] = train_labels.index
train_labels = train_labels.drop_duplicates(['IMAGE_NAME2'])

## use a copy on the local drive to make testing faster
mask_dir = "D:/mammography/mask/train_png/Calc"
img_dir = "D:/mammography/full/train_png/Calc_clahe"

roi_sizes = []

train_calc_patch, train_calc_Lbl, train_calc_FN, train_calc_roi_size = \
    create_patches(roi_sizes, mask_dir, img_dir, Lbls=train_labels, debug=True)

  0%|                                                                                 | 1/1546 [00:00<06:18,  4.08it/s]

(2761, 5056)
Mask Calc-Training_P_00005_RIGHT_CC_1.png  Height: 549 Width: 637
start_row 2431
end_row 3455
start_col 1391
end_col 2415
roi_img.shape (1024, 1024)


  0%|                                                                                 | 2/1546 [00:00<06:19,  4.07it/s]

(2836, 5386)
Mask Calc-Training_P_00005_RIGHT_MLO_1.png  Height: 509 Width: 641
start_row 3036
end_row 4060
start_col 1348
end_col 2372
roi_img.shape (1024, 1024)


  0%|▏                                                                                | 3/1546 [00:00<06:12,  4.14it/s]

...

Mask Calc-Training_P_02572_LEFT_MLO_1.png  Height: 481 Width: 573
start_row 4832
end_row 5856
start_col 1008
end_col 2032
roi_img.shape (1024, 1024)
(4111, 6466)


100%|█████████████████████████████████████████████████████████████████████████████▉| 1545/1546 [06:18<00:00,  3.07it/s]

Mask Calc-Training_P_02584_LEFT_CC_1.png  Height: 489 Width: 525
start_row 4106
end_row 5130
start_col 0
end_col 1024
roi_img.shape (1024, 1024)
(4471, 6361)


100%|██████████████████████████████████████████████████████████████████████████████| 1546/1546 [06:19<00:00,  4.08it/s]

Mask Calc-Training_P_02584_LEFT_MLO_1.png  Height: 465 Width: 517
start_row 4127
end_row 5151
start_col 0
end_col 1024
roi_img.shape (1024, 1024)


In [6]:
print("Train calc patches shape:", train_calc_patch.shape)
print("Train calc Labels:", len(train_calc_Lbl))
print("Train calc File Name:", len(train_calc_FN))

Train calc patches shape: (1546, 256, 256, 1)
Train calc Labels: 1546
Train calc File Name: 1546


In [7]:
np.save(os.path.join("npy_new4", "train_calc_patch.npy"), train_calc_patch)
np.save(os.path.join("npy_new4", "train_calc_Lbl.npy"), np.array(train_calc_Lbl))
np.save(os.path.join("npy_new4", "train_calc_FN.npy"), train_calc_FN)
np.save(os.path.join("npy_new4", "train_calc_roi_size.npy"), np.array(train_calc_roi_size))

In [8]:
#Mass_training
train_labels = pd.read_pickle("label old ver/train_label.pkl")
train_labels['IMAGE_NAME2'] = train_labels.index
train_labels = train_labels.drop_duplicates(['IMAGE_NAME2'])

## use a copy on the local drive to make testing faster
mask_dir = "D:/mammography/mask/train_png/Mass"
img_dir = "D:/mammography/full/train_png/Mass_clahe"

roi_sizes = []

train_mass_patch, train_mass_Lbl, train_mass_FN, train_mass_roi_size = \
    create_patches(roi_sizes, mask_dir, img_dir, Lbls=train_labels, debug=True)

  0%|                                                                                 | 1/1318 [00:00<06:23,  3.43it/s]

(3024, 4808)
Mask Mass-Training_P_00001_LEFT_CC_1.png  Height: 475 Width: 391
start_row 2147
end_row 3171
start_col 0
end_col 1024
roi_img.shape (1024, 1024)


  0%|                                                                                 | 2/1318 [00:00<06:15,  3.50it/s]

(2656, 4800)
Mask Mass-Training_P_00001_LEFT_MLO_1.png  Height: 252 Width: 216
start_row 2594
end_row 3618
start_col 0
end_col 1024
roi_img.shape (1024, 1024)


  0%|▏                                                                                | 3/1318 [00:00<06:25,  3.41it/s]

...

Mask Mass-Training_P_02079_RIGHT_MLO_1.png  Height: 293 Width: 329
start_row 2044
end_row 3068
start_col 987
end_col 2011
roi_img.shape (1024, 1024)
(2585, 4556)


100%|█████████████████████████████████████████████████████████████████████████████▉| 1317/1318 [06:12<00:00,  4.17it/s]

Mask Mass-Training_P_02092_LEFT_CC_1.png  Height: 280 Width: 242
start_row 2436
end_row 3460
start_col 0
end_col 1024
roi_img.shape (1024, 1024)
(3042, 4647)


100%|██████████████████████████████████████████████████████████████████████████████| 1318/1318 [06:12<00:00,  3.53it/s]

Mask Mass-Training_P_02092_LEFT_MLO_1.png  Height: 231 Width: 312
start_row 2688
end_row 3712
start_col 296
end_col 1320
roi_img.shape (1024, 1024)


In [9]:
print("Train mass patches shape:", train_mass_patch.shape)
print("Train mass Labels:", len(train_mass_Lbl))
print("Train mass File Name:", len(train_mass_FN))

Train mass patches shape: (1318, 256, 256, 1)
Train mass Labels: 1318
Train mass File Name: 1318


In [10]:
np.save(os.path.join("npy_new4", "train_mass_patch.npy"), train_mass_patch)
np.save(os.path.join("npy_new4", "train_mass_Lbl.npy"), np.array(train_mass_Lbl))
np.save(os.path.join("npy_new4", "train_mass_FN.npy"), train_mass_FN)
np.save(os.path.join("npy_new4", "train_mass_roi_size.npy"), np.array(train_mass_roi_size))

In [11]:
def create_patches(roi_size, mask_dir, img_dir, Lbls, size=256, debug=True):
    patch_list = []
    Lbl_list = []
    FN_list = []
    
    full_size = 512
    masks = os.listdir(mask_dir)
    counter = 0
    if debug is None:
        progress(counter, len(masks), 'WORKING')
    for mask in tqdm(masks):
        counter += 1
        if debug is None:
            progress(counter, len(masks), mask)
        base_img_file = mask[:-6] + ".png"
        full_img = PIL.Image.open(img_dir+"/"+base_img_file)
#        try:
#            full_img = PIL.Image.open(img_dir + "/" + base_img_file + '.png')
#        except:
#            try:
#                full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000000.png')
#            except:
#                try:
#                    full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000001.png')
#                except:
#                    print("Error FileNotFound:", base_img_file)
#                    continue
        compare = base_img_file[10:]
        #print(compare)
        try:
            Lbl = Lbls.loc[compare]['Class']
        except:
            print("Error LabelNotFound", base_img_file)
            continue
            
        full_img_arr = np.array(full_img)[:,:]
        ctr_row, ctr_col, too_big, mask_size = mask_img2(mask_dir + "/" + mask,full_img_arr)
        img_h, img_w = full_img_arr.shape
        try:
            mask_H = mask_size[0]
            mask_W = mask_size[1]
            roi_size = np.max([mask_H, mask_W])
            if debug:
                print("Mask", mask, " Height:", mask_H, "Width:", mask_W)
        except:
            print("Mask Size Error:", mask_size, "for", mask)
        # Record roi size for DDSM image crop
        roi_sizes.append(roi_size)
        if (ctr_row == 0) and (ctr_col == 0):
            print("Error, skipping", mask)
            continue
        """
        Extract the ROI depending on it's size
        If the ROI is smaller than a slice extract it with some padding
        """
        
#        if roi_size < full_size:

        if(mask_H > mask_W):
            mask_W = mask_H
        else:
            mask_H = mask_W
        
        adj_mask_H = mask_H*1.15
        adj_mask_W = mask_W*1.15
        
        
        start_row = np.max([ctr_row - 512, 0])
        if start_row == 0 :
            end_row = 1024
        else :
            end_row = start_row + 1024
            
        if end_row > img_h:
            end_row = img_h
            start_row = img_h - 1024
            
        start_col = np.max([ctr_col - 512, 0])
        if start_col == 0:
            end_col = 1024
        else :
            end_col = start_col + 1024
        if end_col > img_w:
            end_col = img_w
            start_col = img_w - 1024
       
        end_row = int(end_row)
        end_col = int(end_col)
        start_row = int(start_row)
        start_col = int(start_col)
        print("start_row",start_row)
        print("end_row",end_row)
        print("start_col",start_col)
        print("end_col",end_col)
        
        roi_img = random_flip_img_train(full_img_arr[start_row:end_row, start_col:end_col])
        
        print("roi_img.shape",roi_img.shape)    
        
        roi_img = roi_img.astype('float32')
        
        roi_img = cv2.resize(roi_img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
        roi_img = roi_img.reshape((256, 256, 1))
        #print(roi_img.shape[0])
        #print(roi_img.shape[1])
        
        
        #print(roi_img)
#        patch_1 = crop_img2(roi_img)
#        patch_2 = crop_img2(roi_img)
#        patch_3 = crop_img2(roi_img)
        
        patch_list.append(roi_img)
        Lbl_list.append(Lbl)
        FN_list.append(base_img_file) 
            
#        if (patch_1.shape[0] == size) and (patch_1.shape[1] == size):
#            patch_list.append(patch_1)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
#        if (patch_2.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_2)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
        
#        if (patch_3.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_3)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
    return np.array(patch_list), np.array(Lbl_list), np.array(FN_list), roi_sizes

def create_patches(roi_size, mask_dir, img_dir, Lbls, size=256, debug=True):
    patch_list = []
    Lbl_list = []
    FN_list = []
    
    full_size = 512
    masks = os.listdir(mask_dir)
    counter = 0
    if debug is None:
        progress(counter, len(masks), 'WORKING')
    for mask in tqdm(masks):
        counter += 1
        if debug is None:
            progress(counter, len(masks), mask)
        base_img_file = mask[:-6] + ".png"
        full_img = PIL.Image.open(img_dir+"/"+base_img_file)
#        try:
#            full_img = PIL.Image.open(img_dir + "/" + base_img_file + '.png')
#        except:
#            try:
#                full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000000.png')
#            except:
#                try:
#                    full_img = PIL.Image.open(img_dir + "/" + base_img_file + '000001.png')
#                except:
#                    print("Error FileNotFound:", base_img_file)
#                    continue
        compare = base_img_file[10:]
        #print(compare)
        try:
            Lbl = Lbls.loc[compare]['Class']
        except:
            print("Error LabelNotFound", base_img_file)
            continue
            
        full_img_arr = np.array(full_img)[:,:]
        ctr_row, ctr_col, too_big, mask_size = mask_img2(mask_dir + "/" + mask,full_img_arr)
        img_h, img_w = full_img_arr.shape
        try:
            mask_H = mask_size[0]
            mask_W = mask_size[1]
            roi_size = np.max([mask_H, mask_W])
            if debug:
                print("Mask", mask, " Height:", mask_H, "Width:", mask_W)
        except:
            print("Mask Size Error:", mask_size, "for", mask)
        # Record roi size for DDSM image crop
        roi_sizes.append(roi_size)
        if (ctr_row == 0) and (ctr_col == 0):
            print("Error, skipping", mask)
            continue
        """
        Extract the ROI depending on it's size
        If the ROI is smaller than a slice extract it with some padding
        """
        
#        if roi_size < full_size:

        if(mask_H > mask_W):
            mask_W = mask_H
        else:
            mask_H = mask_W
       
        adj_mask_H = mask_H*1.15
        adj_mask_W = mask_W*1.15
        
        
        start_row = np.max([ctr_row - (adj_mask_H // 2), 0])
        end_row = start_row + adj_mask_H
        if end_row > img_h:
            end_row = img_h
            start_row = img_h - adj_mask_H
            
        start_col = np.max([ctr_col - (adj_mask_W // 2), 0])
        end_col = start_col + adj_mask_W
        if end_col > img_w:
            end_col = img_w
            start_col = img_w - adj_mask_W
        
        end_row = int(end_row)
        end_col = int(end_col)
        start_row = int(start_row)
        start_col = int(start_col)
        roi_img = random_flip_img_train(full_img_arr[start_row:end_row, start_col:end_col])
#        roi_img = random_flip_img_train(full_img_arr[0:img_h, 0:img_w])
#        roi_img = full_img_arr
        print(roi_img.shape)    
        
        roi_img = roi_img.astype('float32')
        
        roi_img = cv2.resize(roi_img, dsize=(1024, 1024), interpolation=cv2.INTER_CUBIC)
        roi_img = roi_img.reshape((1024, 1024, 1))  
#        roi_img = cv2.resize(roi_img, dsize=(400, 400), interpolation=cv2.INTER_CUBIC)
#        roi_img = roi_img.reshape((400, 400, 1))
        #print(roi_img.shape[0])
        #print(roi_img.shape[1])
        
        
        #print(roi_img)
#        patch_1 = crop_img2(roi_img)
#        patch_2 = crop_img2(roi_img)
#        patch_3 = crop_img2(roi_img)
        
        patch_list.append(roi_img)
        Lbl_list.append(Lbl)
        FN_list.append(base_img_file) 
            
#        if (patch_1.shape[0] == size) and (patch_1.shape[1] == size):
#            patch_list.append(patch_1)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
#        if (patch_2.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_2)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
        
#        if (patch_3.shape[0] == size) and (patch_2.shape[1] == size):
#            patch_list.append(patch_3)
#            Lbl_list.append(Lbl)
#            FN_list.append(base_img_file)
                
    return np.array(patch_list), np.array(Lbl_list), np.array(FN_list), roi_sizes

In [12]:
#Calc_test
test_labels = pd.read_pickle("label old ver/test_label.pkl")
test_labels['IMAGE_NAME2'] = test_labels.index
test_labels = test_labels.drop_duplicates(['IMAGE_NAME2'])

## use a copy on the local drive to make testing faster
mask_dir = "D:/mammography/mask/test_png/Calc"
img_dir = "D:/mammography/full/test_png/Calc_clahe"

roi_sizes = []

test_calc_patch, test_calc_Lbl, test_calc_FN, test_calc_roi_size = \
    create_patches(roi_sizes, mask_dir, img_dir, Lbls=test_labels, debug=True)

  0%|▎                                                                                 | 1/326 [00:00<01:40,  3.25it/s]

(3016, 4616)
Mask Calc-Test_P_00038_LEFT_CC_1.png  Height: 209 Width: 225
start_row 1568
end_row 2592
start_col 0
end_col 1024
roi_img.shape (1024, 1024)


  1%|▌                                                                                 | 2/326 [00:00<01:39,  3.24it/s]

(3064, 4728)
Mask Calc-Test_P_00038_LEFT_MLO_1.png  Height: 225 Width: 209
start_row 3416
end_row 4440
start_col 152
end_col 1176
roi_img.shape (1024, 1024)


  1%|▊                                                                                 | 3/326 [00:00<01:36,  3.34it/s]

...

Mask Calc-Test_P_02498_RIGHT_MLO_1.png  Height: 357 Width: 397
start_row 3368
end_row 4392
start_col 789
end_col 1813
roi_img.shape (1024, 1024)
(2311, 4726)


100%|███████████████████████████████████████████████████████████████████████████████▊| 325/326 [01:40<00:00,  3.00it/s]

Mask Calc-Test_P_02501_RIGHT_CC_1.png  Height: 601 Width: 353
start_row 694
end_row 1718
start_col 1287
end_col 2311
roi_img.shape (1024, 1024)
(2371, 5251)


100%|████████████████████████████████████████████████████████████████████████████████| 326/326 [01:41<00:00,  3.23it/s]

Mask Calc-Test_P_02501_RIGHT_MLO_1.png  Height: 617 Width: 593
start_row 1947
end_row 2971
start_col 1099
end_col 2123
roi_img.shape (1024, 1024)


In [13]:
print("Test calc patches shape:", test_calc_patch.shape)
print("Test calc Labels:", len(test_calc_Lbl))
print("Test calc File Name:", len(test_calc_FN))

Test calc patches shape: (326, 256, 256, 1)
Test calc Labels: 326
Test calc File Name: 326


In [14]:
np.save(os.path.join("npy_new4", "test_calc_patch.npy"), test_calc_patch)
np.save(os.path.join("npy_new4", "test_calc_Lbl.npy"), np.array(test_calc_Lbl))
np.save(os.path.join("npy_new4", "test_calc_FN.npy"), test_calc_FN)
np.save(os.path.join("npy_new4", "test_calc_roi_size.npy"), np.array(test_calc_roi_size))

In [15]:
#Mass_test
test_labels = pd.read_pickle("label old ver/test_label.pkl")
test_labels['IMAGE_NAME2'] = test_labels.index
test_labels = test_labels.drop_duplicates(['IMAGE_NAME2'])

## use a copy on the local drive to make testing faster
mask_dir = "D:/mammography/mask/test_png/Mass"
img_dir = "D:/mammography/full/test_png/Mass_clahe"

roi_sizes = []

test_mass_patch, test_mass_Lbl, test_mass_FN, test_mass_roi_size = \
    create_patches(roi_sizes, mask_dir, img_dir, Lbls=test_labels, debug=True)

  0%|▏                                                                                 | 1/378 [00:00<01:10,  5.37it/s]

(1846, 4006)
Mask Mass-Test_P_00016_LEFT_CC_1.png  Height: 344 Width: 345
start_row 2248
end_row 3272
start_col 822
end_col 1846
roi_img.shape (1024, 1024)
(2011, 5491)


  1%|▍                                                                                 | 2/378 [00:00<01:14,  5.05it/s]

Mask Mass-Test_P_00016_LEFT_MLO_1.png  Height: 329 Width: 288
start_row 3311
end_row 4335
start_col 987
end_col 2011
roi_img.shape (1024, 1024)


  1%|▋                                                                                 | 3/378 [00:00<01:31,  4.11it/s]

...

(2281, 4711)
Mask Mass-Test_P_01865_LEFT_MLO_1.png  Height: 168 Width: 305
start_row 2374
end_row 3398
start_col 1166
end_col 2190
roi_img.shape (1024, 1024)
(2041, 3946)


100%|███████████████████████████████████████████████████████████████████████████████▊| 377/378 [01:56<00:00,  4.28it/s]

Mask Mass-Test_P_01912_RIGHT_CC_1.png  Height: 406 Width: 370
start_row 1708
end_row 2732
start_col 1017
end_col 2041
roi_img.shape (1024, 1024)
(2386, 5011)
Mask Mass-Test_P_01912_RIGHT_MLO_1.png  Height: 319 Width: 348
start_row 2144
end_row 3168

100%|████████████████████████████████████████████████████████████████████████████████| 378/378 [01:56<00:00,  3.24it/s]


start_col 1237
end_col 2261
roi_img.shape (1024, 1024)


In [16]:
np.save(os.path.join("npy_new4", "test_mass_patch.npy"), test_mass_patch)
np.save(os.path.join("npy_new4", "test_mass_Lbl.npy"), np.array(test_mass_Lbl))
np.save(os.path.join("npy_new4", "test_mass_FN.npy"), test_mass_FN)
np.save(os.path.join("npy_new4", "test_mass_roi_size.npy"), np.array(test_mass_roi_size))

In [17]:
#데이터 합치기
train_patch = np.concatenate([train_mass_patch, train_calc_patch], axis=0)
train_Lbl = np.concatenate([train_mass_Lbl, train_calc_Lbl], axis=0)
train_FN = np.concatenate([train_mass_FN, train_calc_FN], axis=0)

test_patch = np.concatenate([test_mass_patch, test_calc_patch], axis=0)
test_Lbl = np.concatenate([test_mass_Lbl, test_calc_Lbl], axis=0)
test_FN = np.concatenate([test_mass_FN, test_calc_FN], axis=0)

In [18]:
np.save(os.path.join("abnormal_new4", "abnormal_train_patch.npy"), train_patch)
np.save(os.path.join("abnormal_new4", "abnormal_train_Lbl.npy"), train_Lbl)
np.save(os.path.join("abnormal_new4", "abnormal_train_FN.npy"), train_FN)

np.save(os.path.join("abnormal_new4", "abnormal_test_patch.npy"), test_patch)
np.save(os.path.join("abnormal_new4", "abnormal_test_Lbl.npy"), test_Lbl)
np.save(os.path.join("abnormal_new4", "abnormal_test_FN.npy"), test_FN)

In [19]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline

In [20]:
#train data
CBIS_train_patches = np.load(os.path.join("abnormal_new4", "abnormal_train_patch.npy" ))
CBIS_train_labels = np.load(os.path.join("abnormal_new4", "abnormal_train_Lbl.npy" ))
CBIS_train_FNs = np.load(os.path.join("abnormal_new4", "abnormal_train_FN.npy" ))

#test data
CBIS_test_patches = np.load(os.path.join("abnormal_new4", "abnormal_test_patch.npy" ))
CBIS_test_labels = np.load(os.path.join("abnormal_new4", "abnormal_test_Lbl.npy" ))
CBIS_test_FNs = np.load(os.path.join("abnormal_new4", "abnormal_test_FN.npy" ))



print("Abnaormal train Patches:", CBIS_train_patches.shape)
print("Abnaormal train Labels:", CBIS_train_labels.shape)
print("Abnaormal train File Names:", CBIS_train_FNs.shape)
print("\n")
print("Abnaormal test Patches:", CBIS_test_patches.shape)
print("Abnaormal test Labels:", CBIS_test_labels.shape)
print("Abnaormal test File Names:", CBIS_test_FNs.shape)

Abnaormal train Patches: (2864, 256, 256, 1)
Abnaormal train Labels: (2864,)
Abnaormal train File Names: (2864,)


Abnaormal test Patches: (704, 256, 256, 1)
Abnaormal test Labels: (704,)
Abnaormal test File Names: (704,)


In [21]:
#combine train and test data 

CBIS_all_patches = np.concatenate([CBIS_train_patches, CBIS_test_patches], axis=0)
CBIS_all_labels = np.concatenate([CBIS_train_labels, CBIS_test_labels], axis=0)
CBIS_all_FNs = np.concatenate([CBIS_train_FNs, CBIS_test_FNs], axis=0)

CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs = \
shuffle(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, random_state=19510705)

In [22]:
#split the combined data into train and test
train_patches, test_patches, train_labels, test_labels, train_FNs, test_FNs = \
train_test_split(CBIS_all_patches, CBIS_all_labels, CBIS_all_FNs, test_size = 0.183565, random_state=19430727)

In [23]:
#train data
train_images = train_patches
train_labels = train_labels
train_FNs = train_FNs

#test data
test_images = test_patches
test_labels = test_labels
test_FNs = test_FNs

In [24]:
test_images.shape

(655, 256, 256, 1)

In [25]:
le = preprocessing.LabelEncoder()
le.fit(train_labels)

LabelEncoder()

In [26]:
list(le.classes_)

['BENIGN_calcification',
 'BENIGN_mass',
 'MALIGNANT_calcification',
 'MALIGNANT_mass']

In [27]:
#Convert Normal to 0 
train_labels_en = le.transform(train_labels)
#train_labels_en[train_labels_en==]=0

test_labels_en = le.transform(test_labels)
#test_labels_en[test_labels_en==5]=0

In [28]:
classes = le.classes_

In [29]:
train_bin_labels = np.zeros(len(train_labels_en)).astype(np.int32)
train_bin_labels[train_labels_en != 0] = 1

test_bin_labels = np.zeros(len(test_labels_en)).astype(np.int32)
test_bin_labels[test_labels_en != 0] = 1

In [30]:
train_labels_en

array([0, 0, 0, ..., 2, 3, 2], dtype=int64)

In [31]:
np.save(os.path.join("Label4", "train_labels_en.npy"), train_labels_en)
np.save(os.path.join("Label4", "test_labels_en.npy"), test_labels_en)
np.save(os.path.join("Label4", "train_bin_labels.npy"), train_bin_labels)
np.save(os.path.join("Label4", "test_bin_labels.npy"), test_bin_labels)

In [32]:
#X_val, X_test, y_val, y_test, y_val_multi, y_test_multi = \
#    train_test_split(test_images, test_bin_labels, test_labels_en, test_size=0.5, random_state=19730104)
X_train, y_train, y_train_multi = \
     shuffle(train_images, train_bin_labels, train_labels_en, random_state=100)
X_test, y_test, y_test_multi = \
     shuffle(test_images, test_bin_labels, test_labels_en, random_state=100)

In [33]:
y_test_multi

array([2, 0, 2, 0, 1, 1, 0, 2, 0, 3, 1, 3, 1, 3, 2, 0, 0, 1, 0, 1, 2, 2,
       1, 2, 2, 1, 1, 3, 0, 1, 3, 0, 2, 1, 3, 3, 1, 0, 3, 0, 3, 2, 1, 0,
       0, 3, 0, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 3, 3, 1, 3, 0, 1, 3, 1, 1,
       2, 2, 3, 1, 1, 0, 1, 0, 2, 1, 2, 0, 1, 0, 1, 1, 3, 2, 3, 3, 2, 0,
       3, 1, 0, 0, 0, 2, 0, 0, 3, 2, 2, 0, 1, 0, 0, 1, 1, 1, 0, 3, 0, 0,
       0, 3, 3, 2, 2, 2, 0, 3, 3, 0, 2, 1, 3, 1, 2, 2, 0, 0, 3, 1, 1, 2,
       1, 0, 3, 0, 2, 0, 2, 2, 0, 3, 3, 3, 1, 0, 2, 0, 0, 0, 0, 1, 0, 3,
       2, 3, 0, 1, 2, 0, 1, 0, 1, 3, 2, 0, 0, 0, 0, 2, 3, 3, 1, 0, 3, 3,
       3, 1, 2, 3, 3, 1, 1, 0, 1, 2, 1, 0, 3, 0, 0, 0, 3, 2, 1, 1, 1, 3,
       0, 2, 2, 2, 0, 0, 2, 0, 3, 2, 1, 0, 3, 1, 2, 0, 0, 3, 2, 1, 2, 1,
       1, 3, 2, 1, 0, 3, 3, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 1,
       0, 0, 1, 0, 2, 3, 3, 0, 1, 0, 3, 2, 3, 0, 2, 2, 3, 0, 0, 2, 1, 2,
       3, 1, 2, 1, 0, 1, 3, 0, 1, 2, 1, 3, 0, 0, 0, 1, 2, 1, 2, 3, 0, 0,
       3, 0, 3, 0, 1, 0, 0, 2, 1, 0, 1, 2, 2, 0, 0,

In [34]:
len(X_train)

2913

In [36]:
np.save(os.path.join("Data_new4", 'X_train.npy'), X_train)
np.save(os.path.join("Data_new4", 'y_train.npy'), y_train)
np.save(os.path.join("Data_new4", 'train_labels_multi.npy'), y_train_multi)

np.save(os.path.join("Data_new4", 'X_test.npy'), X_test)
np.save(os.path.join("Data_new4", 'y_test.npy'), y_test)
np.save(os.path.join("Data_new4", 'y_test_labels_multi.npy'), y_test_multi)